### Introduction & Notebook Objectives

### Setup & Install Dependencies

In [ ]:
#Install Dependencies
!pip install pandas numpy scikit-learn nltk matplotlib seaborn

In [17]:
#pull from get
!git fetch origin
!git reset --hard origin/main

From https://github.com/Sorrel-Alvason/cs-ticket-categorizer
   cb6e5ee..6e8590a  main       -> origin/main


HEAD is now at 6e8590a Add files via upload


In [18]:
#push to git
!git status
!git add 01_data_preprocessing.ipynb
!git commit -m "Edited Data Preprocessing Notebook"
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Everything up-to-date


### Load the Dataset

#### Overview
In this Section, we will: 
    Load the dataset from 'dataset-tickets-milti-lang-4-20k.csv'
    Display the first few rows to understand its structure. 
    Filter out the necessary columns ('subject', 'body', 'queue', 'priority')
    check for missing values and basic statistics. 

#### Dataset Description
The dataset consists of customer service tickets with the following columns:
- **`subject`**: The subject line of the ticket (short summary).
- **`body`**: The detailed content of the ticket.
- **`queue`**: The category or type of issue the ticket belongs to.
- **`priority`**: The urgency level of the ticket.

These will be the key inputs (**features**) and outputs (**labels**) for our machine learning model.

In [13]:
import os

print("Current Working Directory:", os.getcwd())  # Print the current path
print("Folders in Current Directory:", os.listdir())  # List folders/files in the current directory


Current Working Directory: C:\Users\Sorrel\Documents\School folder\Portfolio\Ticket Categorizer\cs-ticket-categorizer\notebooks
Folders in Current Directory: ['.gitkeep', '.ipynb_checkpoints', '01_data_preprocessing.ipynb', '02_feature_engineering.ipynb', '03_model_training.ipynb', '04_model_evaluation.ipynb']


In [12]:
import os

# List files in the data directory
print(os.listdir("data/"))


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/'

In [15]:
import pandas as pd

# Load dataset

df = pd.read_csv(r"C:\Users\Sorrel\Documents\School folder\Portfolio\Ticket Categorizer\data\dataset-tickets-multi-lang-4-20k.csv")

# Display basic info
print("Dataset Shape:", df.shape)
print("Column Names:", df.columns.tolist())

# Select only relevant columns
df = df[['subject', 'body', 'queue', 'priority']]

# Show first few rows
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Sorrel\\Documents\\School folder\\Portfolio\\Ticket Categorizer\\data\\dataset-tickets-multi-lang-4-20k.csv'

### Explore and Understand Data

### Clean and Preprocess Text Data

### Tokenization

### Stemming/lemmatization

### Saving the Cleaned Dataset